# Setup

Import openai and set your API key.

In [ ]:
import openai
import os
import json
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Helper para obtener el texto de una url

In [ ]:
def get_website_text(url):
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    return soup.get_text('\n')

In [ ]:
text = get_website_text('https://rootstrap.com')
print(text)

# Definir función y JSON Schema

In [ ]:
def print_company_information(name, contact_email, industry, services):
    print(f"Company Name: {name}")
    print(f"Contact Email: {contact_email}")
    print(f"Industry: {industry}")
    print(f"Services: {services}")


print_company_schema = [
    {
        "name": "print_company_information",
        "description": "Muestra en pantalla la información de una compañía",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "El nombre de la compañía"
                },
                "contact_email": {
                    "type": "string",
                    "format": "email",
                    "description": "El email de contacto de la compañía"
                },
                "industry": {
                    "type": "string",
                    "description": "La industria de la compañía",
                    "enum": ['tech', 'finance', 'healthcare', 'education', 'retail', 'other']
                },
                "services": {
                    "type": "array",
                    "description": "Los servicios que ofrece la compañía",
                    "items": {
                        "type": "string"
                    },

                }
            },
            "required": ["name", "contact_email", "industry", "services"]
        },
    }
]

In [ ]:
def get_openai_response(text):
    response = openai.ChatCompletion.create(
        model='gpt-4-0613',
        functions=print_company_schema,
        function_call={"name": "print_company_information"},
        messages=[
            {
                "role": "system",
                "content": """
                    Eres un transformador de texto sin procesar a estructura de datos.
                    El mensaje del usuario es el texto sin procesar de un sitio web una empresa.
                    Tu tarea es extraer del texto la información de la empresa y llamar a la función print_company_information.
                """
            },
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message

In [ ]:
text = get_website_text("https://rootstrap.com/")
print(text)
response = get_openai_response(text)
print(response)
arguments = json.loads(response.function_call.arguments)
print(arguments)
print_company_information(**arguments)

# Generar propiedades

In [ ]:
tag_message_schema = [
    {
        "name": "tag_message",
        "description": "Etiqueta el mensaje con el sentimiento, tono y código de idioma",
        "parameters": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "string",
                    "description": "El sentimiento del mensaje"
                },
                "tone": {
                    "type": "string",
                    "description": "El tono del mensaje"
                },
                "language_code": {
                    "type": "string",
                    "description": "El código ISO 639-1 del idioma del mensaje"
                },
                "culture": {
                    "type": "string",
                    "description": "La cultura de quien escribe el mensaje"
                }
            },
            "required": ["sentiment", "tone", "language_code"]
        },
    }
]

def tag_message(text):
    response = openai.ChatCompletion.create(
        model='gpt-4-0613',
        functions=tag_message_schema,
        function_call={"name": "tag_message"},
        messages=[
            {
                "role": "system",
                "content": """
                    Eres un etiquetador de mensajes.
                    El mensaje del usuario es el mensaje que debes etiquetar.
                    Tu objetivo es clasificar el mensaje y llamar a la función tag_message.
                """
            },
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.function_call

In [ ]:
function_call = tag_message("I am very happy today")
print(function_call.name)
print(json.loads(function_call.arguments))